# [STEP1] Embeddingによるクエリに関連するデータベーステーブル・カラムの選定

ユーザーからのクエリ(INPUT)をEmbeddingし、事前処理したテーブルカラムメタデータと比較することで、関連カラムのみを選定します。

In [ ]:
# Pythonライブラリインストール
!python --version
!pip install psycopg2-binary
!pip install python-dotenv
!pip install --upgrade openai
!pip install openai[datalib]

!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install plotly
!pip install scikit-learn
!pip install sqlalchemy


## 環境変数
supabase接続用URL,APIキーと、openai api接続用のAPIキーを設定します。
自身のopenaiアカウントからapi keyを取得してください。

https://platform.openai.com/account/api-keys

supabaseの情報は管理者にお尋ねください。

下記の例では、.envファイルに変数を書き込んで、JupiterNotebookで読み込む仕様で実装しております。

※.envファイルの作成が困難、.envファイルから値を読み込めない場合、
　os.getenv("◯◯")部分に変数値を直接書き込んでいただいても動作自体には問題ありません。

In [20]:
# 環境変数
import os
from dotenv import load_dotenv
load_dotenv()

# supabase接続用変数
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")

# OPENAI API KEY
openai_api_key = os.getenv("OPENAI_API_KEY")

print('環境変数読み込み完了')

環境変数読み込み完了


# 処理実行
## [INPUT] クエリを入力, パラメータを設定

In [ ]:
# INPUTクエリ
# 同時にさまざまなパターンで試験を行うため配列形式で保存しています。
input_queries = [
    "What is the projection relationship from the frontal pole with respect to the mouse brain?", # 大まかな脳領域からの投射関係を出力
    "What are the main brain regions that project to the motor area with respect to the mouse brain?", # 特定の脳部位に対し、投射関係を逆に辿った経路をまとめて出力
    "Is there any differences in the projection relationship to the frontal pole with respect to the brains of male and female mice?", # 雌雄のマウスでの投射関係の違いを出力
]

# 選定テーブルカラム数
# いくつのテーブルカラムを出力するかを設定します
column_selection_number = 5

## クエリのEmbeddingメタデータを作成し、テーブルカラムの事前処理済メタデータと比較
openai apiのcosine similarityにてEmbedding間の差を比較 

※TODO: コード作成中

In [ ]:
import psycopg2
from psycopg2.extras import DictCursor
from openai.embeddings_utils import cosine_similarity
import pandas as pd
import numpy as np

# Connect to the database
conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_pass,
    host=db_host,
    port=db_port
)
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
# Open a cursor to perform database operations
cur = conn.cursor(cursor_factory=DictCursor)
cur.execute('SELECT id,name,acronym FROM structures ORDER BY id ASC;')
rows = cur.fetchall()


# Use your API key
openai.api_key = openai_api_key

# Insert OpenAI text embedding model and input
my_model = 'text-embedding-ada-002'
my_input = '<INSERT_INPUT_HERE>'

# Calculate embedding vector for the input using OpenAI Embeddings endpoint
def get_embedding(model: str, text: str) -> list[float]:
    result = openai.Embedding.create(
      model = my_model,
      input = my_input
    )
    return result['data'][0]['embedding']

# Save embedding vector of the input
input_embedding_vector = get_embedding(my_model, my_input)

# Calculate similarity between the input and "facts" from companies_embeddings.csv file which we created before
df = pd.read_csv('companies_embeddings.csv')
df['embedding'] = df['embedding'].apply(eval).apply(np.array)
df['similarity'] = df['embedding'].apply(lambda x: cosine_similarity(x, input_embedding_vector))

# Find the highest similarity value in the dataframe column 'similarity'
highest_similarity = df['similarity'].max()

# If the highest similarity value is equal or higher than 0.9 then print the 'content' with the highest similarity
if highest_similarity >= 0.9:
    fact_with_highest_similarity = df.loc[df['similarity'] == highest_similarity, 'content']
    print(fact_with_highest_similarity)
# Else pass input to the OpenAI Completions endpoint
else:
    response = openai.Completion.create(
      model = 'text-davinci-003',
      prompt = my_input,
      max_tokens = 30,
      temperature = 0
    )
    content = response['choices'][0]['text'].replace('\n', '')
    print(content)
